In [ ]:

import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from category_encoders import OneHotEncoder
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree


In [ ]:
%load_ext sql
%sql sqlite:////home/jovyan/nepal.sqlite

In [ ]:
%%sql
SELECT name
FROM sqlite_schema
WHERE type = "table"

In [ ]:
%%sql
SELECT distinct(district_id)
FROM id_map

In [ ]:
# Task 4.5.2: Calculate the number of observations in the id_map table associated with district 1.
%%sql
SELECT COUNT(*)
FROM id_map
WHERE district_id = 1;

In [ ]:
# Task 4.5.3: Calculate the number of observations in the id_map table associated with district 3.

In [ ]:
%%sql
SELECT COUNT(*)
FROM id_map
WHERE district_id = 3;

In [4]:
# ask 4.5.4: Join the unique building IDs from Kavrepalanchok in id_map, all the columns from building_structure, and the damage_grade column from building_damage, limiting your results to 5 rows. Make sure you rename the building_id column in id_map as b_id and limit your results to the first five rows of the new table.

In [ ]:
%%sql
SELECT distinct(i.building_id) AS b_id,
       s.*,
       d.damage_grade
FROM id_map AS i
JOIN building_structure AS s ON i.building_id = s.building_id
JOIN building_damage AS d ON i.building_id = d.building_id
WHERE i.district_id = 3
LIMIT 5;




In [3]:
# Task 4.5.5: Write a wrangle function that will use the query you created in the previous task to create a DataFrame. In addition your function should:

# Create a "severe_damage" column, where all buildings with a damage grade greater than 3 should be encoded as 1. All other buildings should be encoded at 0.
# Drop any columns that could cause issues with leakage or multicollinearity in your model.

In [ ]:
import sqlite3
import pandas as pd

def wrangle(db_path):
    """
    Wrangle data from an SQLite database to prepare it for analysis.

    Parameters:
    db_path (str): Path to the SQLite database file.

    Returns:
    pd.DataFrame: A DataFrame with processed and cleaned data.
    """
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)

    # Construct the SQL query
    query = """
    SELECT DISTINCT(i.building_id) AS b_id,
           s.*,
           d.damage_grade
    FROM id_map AS i
    JOIN building_structure AS s ON i.building_id = s.building_id
    JOIN building_damage AS d ON i.building_id = d.building_id
    WHERE i.district_id = 3
    """

    # Read query results into a DataFrame
    df = pd.read_sql_query(query, conn, index_col='b_id')

    # Clean the `damage_grade` column to extract numeric values
    df["damage_grade"] = df["damage_grade"].str.extract(r'(\d+)', expand=False).astype(float)

    # Create the `severe_damage` column (binary encoding: 1 if damage_grade > 3, else 0)
    df["severe_damage"] = (df["damage_grade"] > 3).astype(int)

    # Identify columns to drop (any column containing "post_eq" in its name)
    drop_cols = [col for col in df.columns if "post_eq" in col]

    # Add additional columns to drop
    additional_drop_cols = [
        "damage_grade",         # No longer needed after creating `severe_damage`
        "count_floors_pre_eq",  # Presumably unnecessary for analysis
        "building_id"           # Redundant as `b_id` is already used as the index
    ]
    drop_cols.extend(additional_drop_cols)

    # Drop unnecessary columns
    df.drop(columns=drop_cols, inplace=True)

    return df

# Use the wrangle function
df = wrangle("/home/jovyan/nepal.sqlite")
print(df.head())


In [ ]:
df = wrangle("/home/jovyan/nepal.sqlite")
df.head()

In [ ]:
# Plot value counts of `"severe_damage"`
# create bar chart using
# severe damage column which
# contains two classes
df["severe_damage"].value_counts(normalize=True).plot(
    kind="bar", xlabel="Severe Damage", ylabel="Relative Frequency", title="Class Balance"
);
# Don't delete the code below 👇
plt.savefig("images/4-5-6.png", dpi=150)


In [ ]:
# Task 4.5.7: Is there a relationship between the footprint size of a building and the damage it sustained in the earthquake? Use seaborn to create a boxplot that shows the distributions of the "plinth_area_sq_ft" column for both groups in the "severe_damage" column. Label your x-axis "Severe Damage" and y-axis "Plinth Area [sq. ft.]". Use the title "Kavrepalanchok, Plinth Area vs Building Damage".

In [ ]:
# severe_damage: column with 2 groups
# plinth_area_sq_ft: column: footprint size of building
import seaborn as sns
sns.boxplot(x="severe_damage", y="plinth_area_sq_ft", data=df)
plt.xlabel("Severe Damage")
plt.ylabel("Plinth Area [sq. ft.]")
plt.title("Kavrepalanchok, Plinth Area vs Building Damage");
# Don't delete the code below 👇
plt.savefig("images/4-5-7.png", dpi=150)


In [6]:
# Task 4.5.8: Are buildings with certain roof types more likely to suffer severe damage? Create a pivot table of df where the index is "roof_type" and the values come from the "severe_damage" column, aggregated by the mean.

In [ ]:
roof_pivot = pd.pivot_table(
    df, index="roof_type", values="severe_damage", aggfunc=np.mean    # roof_type: column in table
).sort_values(by="severe_damage")
roof_pivot

In [ ]:
# **Task 4.5.9:** Create your feature matrix `X` and target vector `y`. Your target is `"severe_damage"`.

In [ ]:
X = df.drop(columns="severe_damage")    # feature matrix: all columns apart from severe_damage
y = df["severe_damage"]       # target vector
print("X shape:", X.shape)
print("y shape:", y.shape)



In [ ]:
# Task 4.5.10: Divide your dataset into training and validation sets using a randomized split. Your validation set should be 20% of your data.

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(X , y , test_size=0.2 , random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

In [ ]:
# Task 4.5.11: Calculate the baseline accuracy score for your model

In [ ]:
acc_baseline = y_train.value_counts(normalize = True).max()
print("Baseline Accuracy:", round(acc_baseline, 2))

In [ ]:
# Task 4.5.12: Create a model model_lr that uses logistic regression to predict building damage. Be sure to include an appropriate encoder for categorical features.

In [8]:
model_lr = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'),  # Encodes categorical features
    LogisticRegression(max_iter=2000)        # Specify max_iter to suppress warnings
)

In [ ]:
# Task 4.5.13: Calculate training and validation accuracy score for model_lr

In [ ]:
lr_train_acc = accuracy_score(y_train , model_lr.predict(X_train))

lr_val_acc =model_lr.score(X_val , y_val)

print("Logistic Regression, Training Accuracy Score:", lr_train_acc)
print("Logistic Regression, Validation Accuracy Score:", lr_val_acc)

In [ ]:
# Task 4.5.14: Perhaps a decision tree model will perform better than logistic regression, but what's the best hyperparameter value for max_depth? Create a for loop to train and evaluate the model model_dt at all depths from 1 to 15. Be sure to use an appropriate encoder for your model, and to record its training and validation accuracy scores at every depth. The grader will evaluate your validation accuracy scores only.

In [ ]:
depth_hyperparams = range(1, 16)
training_acc = []
validation_acc = []
for d in depth_hyperparams:
    model_dt = make_pipeline(
        OrdinalEncoder(),
        DecisionTreeClassifier(max_depth=d , random_state =42)

    )
    model_dt.fit(X_train, y_train)
    training_acc.append(model_dt.score(X_train , y_train))
    validation_acc.append(model_dt.score(X_val,y_val))


In [ ]:
# Task 4.5.15: Using the values in training_acc and validation_acc, plot the validation curve for model_dt. Label your x-axis "Max Depth" and your y-axis "Accuracy Score". Use the title "Validation Curve, Decision Tree Model", and include a legend

In [ ]:

# Don't delete the code below 👇
plt.savefig("images/4-5-15.png", dpi=150)


In [ ]:
# Task 4.5.16: Build and train a new decision tree model final_model_dt, using the value for max_depth that yielded the best validation accuracy score in your plot above.

In [ ]:
# Validation curve
plt.plot(depth_hyperparams, training_acc, label="Training")
plt.plot(depth_hyperparams, validation_acc, label="validation")
plt.xlabel("Max Depth")
plt.ylabel("Accuracy Score")
plt.title("Validation Curve, Decision Tree Model")
plt.legend();


# build & fit again
final_model_dt = make_pipeline(
    OrdinalEncoder(),
    DecisionTreeClassifier(max_depth=10, random_state=42)
)
# Fit model to training data
final_model_dt.fit(X, y)

In [ ]:
# Task 4.5.17: How does your model perform on the test set? First, read the CSV file "data/kavrepalanchok-test-features.csv" into the DataFrame X_test. Next, use final_model_dt to generate a list of test predictions y_test_pred. Finally, submit your test predictions to the grader to see how your model performs.

# # Tip: Make sure the order of the columns in X_test is the same as in your X_train. Otherwise, it could hurt your model's performance.

In [ ]:
X_test = pd.read_csv("data/kavrepalanchok-test-features.csv", index_col="b_id")
y_test_pred = pd.Series(final_model_dt.predict(X_test))
y_test_pred[:5]